In [ ]:
# This adds the path to import the development version (git repo) of NDI Python
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from ndi import Experiment, DaqSystem, FileNavigator
from ndi.daqreaders import MockReader
from ndi.database import FileSystem as Database, Query as Q

In [3]:
db = Database('../test_exp')

In [4]:
fn = FileNavigator(epoch_file_patterns=['.*\\.wav', '.*\\.txt'], 
                   metadata_file_pattern='.*\\.txt')

In [5]:
ds = DaqSystem(name='myDaq1',
              file_navigator=fn,
              daq_reader=MockReader)
ds2 = DaqSystem(name='myDaq2',
               file_navigator=fn,
               daq_reader=MockReader)

In [6]:
e = Experiment(name='myExperiment3', 
               daq_systems=[ds, ds2])

In [7]:
db.add_experiment(e)

In [8]:
e2 = db.find_by_id(Experiment, e.id)

In [9]:
e2 == e

True

In [10]:
e2 == db.find(Experiment,Q('name') == 'myExperiment3')[0]

True

In [11]:
db.find(Experiment, Q('name') == 'myExperiment2')

[]

In [12]:
ds_results = db.find(DaqSystem, Q('experiment_id') == e.id)

In [13]:
print(ds_results)

[<ndi.daq_system.DaqSystem object at 0x1150e3a60>, <ndi.daq_system.DaqSystem object at 0x1150e3820>]


In [14]:
e.daq_systems[0] == ds_results[0]

False

In [15]:
db.find(Experiment)

In [16]:
db.delete_by_id(Experiment, e.id)

In [17]:
try:
    db.find_by_id(Experiment, e.id)
except FileNotFoundError:
    print('File not found')

File not found


In [18]:
db.delete_many(Experiment)

In [19]:
db.find(Experiment)

[]

In [20]:
try:
    db.add(ds)
except FileExistsError as e:
    print(e)

File '../test_exp/.ndi/daqsystems/90a1ca556083422dadc47185bdc79b57.dat' already exists


In [21]:
db.upsert(ds)

In [22]:
print(ds.name)

myDaq1


In [23]:
db.update_by_id(DaqSystem, ds.id, { 'name': 'daq2' })

In [24]:
ds2 = db.find_by_id(DaqSystem, ds.id)

In [25]:
print(ds2.name)

daq2


In [26]:
try:
    db.add('a')
except TypeError as e:
    print(e)

'a' is not an instance of an 'NDI_Object' child class.
